In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from dgl.nn import EGATConv, EdgeWeightNorm, GraphConv
from dgl.utils import expand_as_pair
import time

In [38]:
encoder_layer = nn.TransformerEncoderLayer(d_model=64, 
                                           nhead=4,
                                           dim_feedforward=128,)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=1)
src = torch.rand(10, 32, 64) # batch, channel, d_model
out = transformer_encoder(src)

avgpool = nn.AdaptiveAvgPool1d(output_size=1)
out = avgpool(out)
out = torch.flatten(out, 1)
out.shape

torch.Size([10, 32])

In [40]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   learn_dgl.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
decoder_layer = nn.TransformerDecoderLayer(d_model=64, 
                                           nhead=8,
                                           dim_feedforward=128,)
transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=1)
memory = torch.rand(10, 32, 64)
tgt = torch.rand(20, 32, 512)
out = transformer_decoder(tgt, memory)

In [30]:
out.shape

torch.Size([10, 32, 64])

In [39]:
for i in avgpool.named_parameters():
    print(i[1].shape)

In [18]:
from train_gpu import inference_new
run_time = time.strftime("%Y.%m.%d-%H-%M-%S", time.localtime())
# param_search(run_time)
from config import Config
from data import part_DATA
from RC import torchRC, EGAT, EGCN
import dgl
from RC import MLP

config = Config()
config.data = 'mnist'
config.train_num = 10000
config.test_num = 2000
config.N_in = 28*28
config.N_out = 10
train_loader, test_loader = part_DATA(config)

model = torchRC(config).to(config.device)
train_rs, train_label = inference_new(model, config, train_loader,)
test_rs, test_label = inference_new(model, config, test_loader,)
train_rs = train_rs[:,1:,:].mean(1)
test_rs = test_rs[:,1:,:].mean(1)

Data Generation Finish
finish inference
finish inference


In [20]:
from train_gpu import train_mlp_readout
config.epoch = 300
mlp = MLP(2*config.N_hid, config.mlp_hid, config.N_out).to(model.device)
train_score, test_score, = train_mlp_readout(model=mlp, 
                                            config=config,
                                            X_train=train_rs,
                                            X_test=test_rs,
                                            y_train=train_label,
                                            y_test=test_label)

[1,300] loss:1.843, train acc:0.4879, test acc:0.6850
[2,300] loss:1.175, train acc:0.7510, test acc:0.7850
[3,300] loss:0.856, train acc:0.8098, test acc:0.8200
[4,300] loss:0.688, train acc:0.8372, test acc:0.8465
[5,300] loss:0.593, train acc:0.8537, test acc:0.8595
[6,300] loss:0.532, train acc:0.8639, test acc:0.8715
[7,300] loss:0.489, train acc:0.8716, test acc:0.8770
[8,300] loss:0.456, train acc:0.8772, test acc:0.8835
[9,300] loss:0.430, train acc:0.8825, test acc:0.8865
[10,300] loss:0.408, train acc:0.8875, test acc:0.8920
[11,300] loss:0.390, train acc:0.8912, test acc:0.8930
[12,300] loss:0.375, train acc:0.8948, test acc:0.8945
[13,300] loss:0.360, train acc:0.8995, test acc:0.8970
[14,300] loss:0.348, train acc:0.9036, test acc:0.8990
[15,300] loss:0.336, train acc:0.9064, test acc:0.9010
[16,300] loss:0.327, train acc:0.9096, test acc:0.9030
[17,300] loss:0.318, train acc:0.9125, test acc:0.9065
[18,300] loss:0.309, train acc:0.9152, test acc:0.9070
[19,300] loss:0.302

In [61]:
v = train_rs[index][:,1:,0:config.N_hid]
# v = v.transpose(1,2).view(-1,30)
# node_feat = Egcn(g, v[0:config.N_hid], edge_attr)

In [76]:
train_label[index].view(1,-1)[0]

tensor([0, 3, 4, 7, 0])

In [69]:
v.shape

torch.Size([5, 30, 200])

In [70]:
batch_v = v.transpose(1,2).reshape(-1, 30)
batch_g = dgl.batch([g]*5)
batch_edge_attr = torch.cat(([edge_attr]*5))

In [71]:
node_feat = Egcn(batch_g, batch_v, batch_edge_attr)
# pred = node_feat.argmax(1)

In [73]:
batch_label

NameError: name 'batch_label' is not defined

In [65]:
pred

tensor([5, 8, 5, 5, 9, 9, 9, 8, 8, 9])

In [20]:
for i in transformer_encoder.named_parameters():
    print(i[1].shape)

torch.Size([1536, 512])
torch.Size([1536])
torch.Size([512, 512])
torch.Size([512])
torch.Size([2048, 512])
torch.Size([2048])
torch.Size([512, 2048])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([1536, 512])
torch.Size([1536])
torch.Size([512, 512])
torch.Size([512])
torch.Size([2048, 512])
torch.Size([2048])
torch.Size([512, 2048])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([1536, 512])
torch.Size([1536])
torch.Size([512, 512])
torch.Size([512])
torch.Size([2048, 512])
torch.Size([2048])
torch.Size([512, 2048])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([1536, 512])
torch.Size([1536])
torch.Size([512, 512])
torch.Size([512])
torch.Size([2048, 512])
torch.Size([2048])
torch.Size([512, 2048])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([1536, 512])
torch.Size([

In [78]:
64*30+64+64*32*32+310+330

68160

In [18]:
batch_f.shape

torch.Size([12, 30])

In [22]:
u = [0,1,2,3,2,5]
v = [1,2,3,4,0,3]
g = dgl.graph((u,v))
g = dgl.add_self_loop(g)

batch_g = dgl.batch([g, g])
feat = torch.ones(6, 3)
batch_f = torch.cat([feat, feat])

edge_weight = torch.tensor([0.5, 0.6, 0.4, 0.7, 0.9, 0.1, 1, 1, 1, 1, 1, 1])
norm = EdgeWeightNorm(norm='both')
norm_edge_weight = norm(g, edge_weight)
conv1 = GraphConv(3, 2, norm='none', weight=True, bias=True, activation=None)
conv2 = GraphConv(3, 2, norm='none', weight=True, bias=True, activation=nn.Softmax())
conv2.weight = conv1.weight
conv2.bias = conv1.bias

# out1 = conv(batch_g, batch_f, edge_weight=norm_edge_weight)
out2 = conv1(batch_g, batch_f, edge_weight=None)
out3 = conv2(batch_g, batch_f, edge_weight=None)
# out3 = conv(g, feat, edge_weight=edge_weight)
# print(out1,'\n', out2, '\n', out3)

c:\ProgramData\Anaconda3\envs\snn\lib\site-packages\dgl\nn\pytorch\conv\graphconv.py:447: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rst = self._activation(rst)


In [23]:
out2,out3

(tensor([[-3.1475,  0.8163],
         [-3.1475,  0.8163],
         [-3.1475,  0.8163],
         [-4.7213,  1.2245],
         [-3.1475,  0.8163],
         [-1.5738,  0.4082],
         [-3.1475,  0.8163],
         [-3.1475,  0.8163],
         [-3.1475,  0.8163],
         [-4.7213,  1.2245],
         [-3.1475,  0.8163],
         [-1.5738,  0.4082]], grad_fn=<AddBackward0>),
 tensor([[0.0186, 0.9814],
         [0.0186, 0.9814],
         [0.0186, 0.9814],
         [0.0026, 0.9974],
         [0.0186, 0.9814],
         [0.1211, 0.8789],
         [0.0186, 0.9814],
         [0.0186, 0.9814],
         [0.0186, 0.9814],
         [0.0026, 0.9974],
         [0.0186, 0.9814],
         [0.1211, 0.8789]], grad_fn=<SoftmaxBackward0>))

In [23]:
a = graph.adjacency_matrix()

TypeError: can't convert Sparse layout tensor to numpy.convert the tensor to a strided layout first.

In [24]:
num_nodes, num_edges = 8, 30
node_dim = 20
edge_dim = 1
graph = dgl.rand_graph(num_nodes, num_edges)
node_feats = torch.rand((num_nodes, node_dim))
edge_feats = torch.rand((num_edges, edge_dim))
egat = EGATConv(in_node_feats=node_dim,
                in_edge_feats=edge_dim,
                out_node_feats=15,
                out_edge_feats=1,
                num_heads=3)
#forward pass
new_node_feats, new_edge_feats = egat(graph, node_feats, edge_feats)
new_node_feats.shape, new_edge_feats.shape

(torch.Size([8, 3, 15]), torch.Size([30, 3, 1]))

In [28]:
new_edge_feats[0]

tensor([[1.7398],
        [2.1219],
        [0.2601]], grad_fn=<SelectBackward0>)

In [29]:
new_edge_feats.mean(1)[0]

tensor([1.3739], grad_fn=<SelectBackward0>)

In [47]:
import dgl.data

dataset = dgl.data.CoraGraphDataset('./data')
print('Number of categories:', dataset.num_classes)
g = dataset[0]
print(g.ndata)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}


In [43]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [13]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(30):
        logits = model(g, features)
        pred = logits.argmax(1)
        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

NameError: name 'GCN' is not defined

In [7]:
from config import Config as config
from RC import torchRC
model = torchRC(config)
mem, spike = model(torch.rand(16, config.frames, 50))
print(mem.shape)

torch.Size([1, 16, 101, 200])


In [45]:
A = model.reservoir.A.numpy()
edge_index = torch.tensor(np.where(A!=0), dtype=torch.long)
edge_attr = torch.tensor(np.array([A[i,j] for i,j in edge_index.T]))
u = edge_index[0]
v = edge_index[1]
mems = mem[0,0,1:].T
g = dgl.graph((u, v))
g.ndata['x'] = mems
g.edata['w'] = edge_attr

h = dgl.graph((u, v))
h.ndata['x'] = mems

In [40]:
g = dgl.graph((u, v))
g.ndata['x'] = mems
g.edata['w'] = edge_attr

h = dgl.graph((u, v))
h.ndata['x'] = mems

In [37]:
from dgl.nn import GraphConv
conv1 = GraphConv(config.frames, 16)
out = conv1(g, mems)
out_h = conv1(h, mems)
print(out_h[0])
print(out[0])

In [47]:
edge_attr

tensor([ 0.3371, -0.0486,  0.0422,  ...,  0.0489, -0.0459,  0.3718])

In [46]:
from torch_geometric.nn import GCNConv
conv2 = GCNConv(100, 16)
conv2(mems, edge_index, edge_attr)

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<AddBackward0>)